In [1]:
import timm
from pprint import pprint
model_names = timm.list_models('edge*', pretrained=True)
pprint(model_names)

['edgenext_small', 'edgenext_small_rw', 'edgenext_x_small', 'edgenext_xx_small']


/home/aiteam/miniconda3/envs/mm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import timm

m = timm.create_model('edgenext_small', pretrained=True)
# m.eval()
# /home/aiteam/.cache/torch/hub/checkpoints/edgenext_small_usi.pth

In [4]:
m.feature_info

[{'num_chs': 48, 'reduction': 4, 'module': 'stages.0'},
 {'num_chs': 96, 'reduction': 8, 'module': 'stages.1'},
 {'num_chs': 160, 'reduction': 16, 'module': 'stages.2'},
 {'num_chs': 304, 'reduction': 32, 'module': 'stages.3'}]

In [2]:
%%capture captured

import mmcv
config = '/home/aiteam/tykim/scratch/lightweight/mmdet_timm/edgenext_exps/faster_rcnn_edgenext_phone.py'
cfg = mmcv.Config.fromfile(config)

In [3]:
cfg.load_from = '/home/aiteam/.cache/torch/hub/checkpoints/edgenext_small_usi.pth'
cfg.gpu_ids = range(1)
cfg.device = 'cuda'

In [4]:
print(f'Config:\n{cfg.pretty_text}')

Config:
checkpoint_config = dict(interval=10)
log_config = dict(
    interval=10,
    hooks=[dict(type='TextLoggerHook'),
           dict(type='TensorboardLoggerHook')])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = '/home/aiteam/.cache/torch/hub/checkpoints/edgenext_small_usi.pth'
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
auto_scale_lr = dict(enable=False, base_batch_size=16)
dataset_type = 'COCODataset'
data_root = '/home/aiteam/tykim/scratch/lightweight/mmcv_phone/hold_smartphone'
img_norm_cfg = dict(
    mean=[103.53, 116.28, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 640), (1333, 800)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
  

In [5]:
# 데이터셋과 detector를 init한다.
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

In [6]:
datasets = [build_dataset(cfg.data.train.dataset)]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [7]:
model = build_detector(cfg.model)

2022-08-23 07:10:14,203 - mmcls - INFO - backbone out_indices: (0, 1, 2, 3)
2022-08-23 07:10:14,204 - mmcls - INFO - backbone out_channels: [48, 96, 160, 304]
2022-08-23 07:10:14,205 - mmcls - INFO - backbone out_strides: [4, 8, 16, 32]


In [8]:
model.CLASSES = datasets[0].CLASSES

In [9]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-08-23 07:10:19,200 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-08-23 07:10:19,210 - mmdet - INFO - load checkpoint from local path: /home/aiteam/.cache/torch/hub/checkpoints/edgenext_small_usi.pth
2022-08-23 07:10:19,232 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: downsample_layers.0.0.weight, downsample_layers.0.0.bias, downsample_layers.0.1.weight, downsample_layers.0.1.bias, downsample_layers.1.0.weight, downsample_layers.1.0.bias, downsample_layers.1.1.weight, downsample_layers.1.1.bias, downsample_layers.2.0.weight, downsample_layers.2.0.bias, downsample_layers.2.1.weight, downsample_layers.2.1.bias, downsample_layers.3.0.weight, downsample_layers.3.0.bias, downsample_layers.3.1.weight, downsample_layers.3.1.bias, stages.0.0.gamma, stages.0.0.dwconv.weight, stages.0.0.dwconv.bias, stages.0.0.norm.weight, stages.0.0.norm.bias, stages.0.0.pwconv1.weight, stages.0.0.pwconv1.bi

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


RuntimeError: CUDA error: an illegal memory access was encountered

In [3]:
from mmdet.apis import set_random_seed

cfg.data_root = '/home/aiteam/tykim/scratch/lightweight/mmcv_phone/hold_smartphone'
# timm이 pretraiend를 로드할 것이므로
cfg.load_from = None
cfg.work_dir = '/home/aiteam/tykim/scratch/lightweight/mmdet_timm/edgenext_exps'

cfg.data.samples_per_gpu=16
cfg.data.workers_per_gpu=4

cfg.optimizer.lr = 0.02 * 1
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 10
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10

cfg.runner.max_epochs = 300

cfg.seed = 0
set_random_seed(0, deterministic=False)

In [25]:
# dump_file = "/home/aiteam/tykim/scratch/lightweight/mmdet_timm/edgenext_exps/faster_rcnn_edgenext_phone.py"
# cfg.dump(dump_file)